In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import glob
print(datetime.datetime.now())
print(os.getcwd())

ty_start_date=datetime.date(2019,11,3)
ty_end_date=datetime.date(2020,2,1)

ly_start_date=ty_start_date-datetime.timedelta(days=52*7)
ly_end_date=ty_end_date-datetime.timedelta(days=52*7)


print("ty_start_date:",ty_start_date)
print("ty_end_date:",ty_end_date)

print("ly_start_date:",ly_start_date)
print("ly_end_date:",ly_end_date)


2020-01-24 14:21:00.676765
/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Planner_Request/JT_non_rewards_sales_so_far_2019Q4_4_weeks
ty_start_date: 2019-11-03
ty_end_date: 2020-02-01
ly_start_date: 2018-11-04
ly_end_date: 2019-02-02


In [2]:
def recursive_file_gen(root_path):
    for root, dirs, files in os.walk(root_path):
        for file in files:
            yield os.path.join(root,file)
list_files_ly=glob.glob("/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/*.txt")
list_files_ly=[x for x in list_files_ly if x.split("/MediaStormDailySalesHistory")[1][:8]>=str(ly_start_date).replace("-","")]
list_files_ly=[x for x in list_files_ly if x.split("/MediaStormDailySalesHistory")[1][:8]<=str(ly_end_date).replace("-","")]
list_files_ly.sort()
print(len(list_files_ly))

13


In [3]:
list_file_ty=recursive_file_gen("/home/jian/BigLots/")
list_file_ty=[x for x in list_file_ty if "dailysales" in x.lower()]
list_file_ty=[x for x in list_file_ty if "/MediaStorm_" in x]
list_file_ty=[x for x in list_file_ty if x.split("/MediaStorm_")[1][:10]>=str(ty_start_date)]
list_file_ty=[x for x in list_file_ty if x.split("/MediaStorm_")[1][:10]<=str(ty_end_date)]
list_file_ty.sort()
print(len(list_file_ty))

11


In [9]:
df_product_taxo=pd.read_table("/home/jian/BigLots/static_files/ProductTaxonomy/MediaStormProductTaxonomy20200101-135600-916.txt",
                              dtype=str,sep="|")
df_product_taxo=df_product_taxo[['division_id','class_code_id','subclass_id']].drop_duplicates()
df_product_taxo['division_id']=df_product_taxo['division_id'].astype(int)
df_division_name=pd.read_table("/home/jian/BigLots/static_files/MediaStorm Data Extract - Division Names.txt",sep="|")
df_product_taxo.head(2)

,division_id,class_code_id,subclass_id
0,1,11001,2
1,1,11001,4


In [29]:
df_summary_InWeek_ly=pd.DataFrame()
df_summary_CumQ_ly=pd.DataFrame()

df_rewards_ids_ly=pd.DataFrame()

cum_R_sales=0
cum_R_trans=0

cum_N_sales=0
cum_N_trans=0

week_num=0
for file in list_files_ly:
    week_num+=1
    df=pd.read_table(file,dtype=str,sep="|",
                    usecols=['location_id','transaction_dt','transaction_id','customer_id_hashed','class_code_id','subclass_id','item_transaction_amt'])
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    df=pd.merge(df,df_product_taxo,on=['class_code_id','subclass_id'],how="left")
    date_min=df['transaction_dt'].min()
    date_max=df['transaction_dt'].max()
    
    df['rewards_label']=np.where(pd.notnull(df['customer_id_hashed']),"Rewards","Non_Rewards")
    df_division=df.groupby(['rewards_label','division_id'])['item_transaction_amt'].sum().to_frame().reset_index()
    df_division_R=df_division[df_division['rewards_label']=="Rewards"].T
    df_division_N=df_division[df_division['rewards_label']=="Non_Rewards"].T
    
    df_division_R_2=df_division_R.loc[df_division_R.index=="item_transaction_amt"]
    df_division_R_2.columns=["Rewards_Div_"+str(x) for x in df_division_R.loc[df_division_R.index=="division_id"].values.tolist()[0]]
    df_division_R=df_division_R_2.reset_index()
    df_division_R['week_num']=week_num

    df_division_N_2=df_division_N.loc[df_division_N.index=="item_transaction_amt"]
    df_division_N_2.columns=["NonRew_Div_"+str(x) for x in df_division_N.loc[df_division_N.index=="division_id"].values.tolist()[0]]
    df_division_N=df_division_N_2.reset_index()
    df_division_N['week_num']=week_num

    del df_division_R['index']
    del df_division_N['index']
    
    df_rewards=df[pd.notnull(df['customer_id_hashed'])]
    df_nonrewards=df[pd.isnull(df['customer_id_hashed'])]
    
    num_R_sales=df_rewards['item_transaction_amt'].sum()
    num_R_trans=df_rewards[['location_id','transaction_dt','transaction_id','customer_id_hashed']].drop_duplicates().shape[0]
    num_R_shoppers=df_rewards['customer_id_hashed'].nunique()
    
    num_N_sales=df_nonrewards['item_transaction_amt'].sum()
    num_N_trans=df_nonrewards[['location_id','transaction_dt','transaction_id']].drop_duplicates().shape[0]
    
    df_rewards_shoppers=df_rewards[['customer_id_hashed']].drop_duplicates()
    df_rewards_shoppers['week_start']=date_min
    df_rewards_shoppers['week_end']=date_max
    df_rewards_ids_ly=df_rewards_ids_ly.append(df_rewards_shoppers)
    
    cum_R_sales+=num_R_sales
    cum_R_trans+=num_R_trans
    cum_N_sales+=num_N_sales
    cum_N_trans+=num_N_trans
    cum_R_shoppers=df_rewards_ids_ly['customer_id_hashed'].nunique()
    
    df_in_week=pd.DataFrame({"week_num":week_num,"week_start":date_min,"week_end":date_max,
                             "Rewards_sales":num_R_sales,"Rewards_trans":num_R_trans,"Rewards_shoppers":num_R_shoppers,
                             'NonRewards_sales':num_N_sales,"NonRewards_trans":num_N_trans,
                            },index=[0])
    
    df_cum_week=pd.DataFrame({"week_num":week_num,"week_start":date_min,"week_end":date_max,
                             "Rewards_sales":cum_R_sales,"Rewards_trans":cum_R_trans,"Rewards_shoppers":cum_R_shoppers,
                             'NonRewards_sales':cum_N_sales,"NonRewards_trans":cum_N_trans,
                            },index=[0])
    df_in_week=pd.merge(df_in_week,df_division_R,on="week_num",how="outer")
    df_in_week=pd.merge(df_in_week,df_division_N,on="week_num",how="outer")
    
    df_summary_InWeek_ly=df_summary_InWeek_ly.append(df_in_week)
    df_summary_CumQ_ly=df_summary_CumQ_ly.append(df_cum_week)
    print(datetime.datetime.now(),week_num,"done")

    
    

2020-01-24 15:06:42.105754 1 done
2020-01-24 15:08:29.194437 2 done
2020-01-24 15:17:37.610658 3 done
2020-01-24 15:30:57.417063 4 done
2020-01-24 15:42:05.186632 5 done
2020-01-24 15:50:11.881793 6 done
2020-01-24 16:00:44.082409 7 done
2020-01-24 16:03:10.884726 8 done
2020-01-24 16:05:05.400245 9 done
2020-01-24 16:09:44.328774 10 done
2020-01-24 16:15:14.390447 11 done
2020-01-24 16:20:34.663750 12 done
2020-01-24 16:24:59.687547 13 done


In [30]:
df_summary_InWeek_ty=pd.DataFrame()
df_summary_CumQ_ty=pd.DataFrame()

df_rewards_ids_ty=pd.DataFrame()

cum_R_sales=0
cum_R_trans=0

cum_N_sales=0
cum_N_trans=0

week_num=0
for file in list_file_ty:
    week_num+=1
    df=pd.read_table(file,dtype=str,sep="|",
                    usecols=['location_id','transaction_dt','transaction_id','customer_id_hashed','class_code_id','subclass_id','item_transaction_amt'])
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    df=pd.merge(df,df_product_taxo,on=['class_code_id','subclass_id'],how="left")
    date_min=df['transaction_dt'].min()
    date_max=df['transaction_dt'].max()
    
    df['rewards_label']=np.where(pd.notnull(df['customer_id_hashed']),"Rewards","Non_Rewards")
    df_division=df.groupby(['rewards_label','division_id'])['item_transaction_amt'].sum().to_frame().reset_index()
    df_division_R=df_division[df_division['rewards_label']=="Rewards"].T
    df_division_N=df_division[df_division['rewards_label']=="Non_Rewards"].T
    
    df_division_R_2=df_division_R.loc[df_division_R.index=="item_transaction_amt"]
    df_division_R_2.columns=["Rewards_Div_"+str(x) for x in df_division_R.loc[df_division_R.index=="division_id"].values.tolist()[0]]
    df_division_R=df_division_R_2.reset_index()
    df_division_R['week_num']=week_num

    df_division_N_2=df_division_N.loc[df_division_N.index=="item_transaction_amt"]
    df_division_N_2.columns=["NonRew_Div_"+str(x) for x in df_division_N.loc[df_division_N.index=="division_id"].values.tolist()[0]]
    df_division_N=df_division_N_2.reset_index()
    df_division_N['week_num']=week_num

    del df_division_R['index']
    del df_division_N['index']
    
    df_rewards=df[pd.notnull(df['customer_id_hashed'])]
    df_nonrewards=df[pd.isnull(df['customer_id_hashed'])]
    
    num_R_sales=df_rewards['item_transaction_amt'].sum()
    num_R_trans=df_rewards[['location_id','transaction_dt','transaction_id','customer_id_hashed']].drop_duplicates().shape[0]
    num_R_shoppers=df_rewards['customer_id_hashed'].nunique()
    
    num_N_sales=df_nonrewards['item_transaction_amt'].sum()
    num_N_trans=df_nonrewards[['location_id','transaction_dt','transaction_id']].drop_duplicates().shape[0]
    
    df_rewards_shoppers=df_rewards[['customer_id_hashed']].drop_duplicates()
    df_rewards_shoppers['week_start']=date_min
    df_rewards_shoppers['week_end']=date_max
    df_rewards_ids_ty=df_rewards_ids_ty.append(df_rewards_shoppers)
    
    cum_R_sales+=num_R_sales
    cum_R_trans+=num_R_trans
    cum_N_sales+=num_N_sales
    cum_N_trans+=num_N_trans
    cum_R_shoppers=df_rewards_ids_ty['customer_id_hashed'].nunique()
    
    df_in_week=pd.DataFrame({"week_num":week_num,"week_start":date_min,"week_end":date_max,
                             "Rewards_sales":num_R_sales,"Rewards_trans":num_R_trans,"Rewards_shoppers":num_R_shoppers,
                             'NonRewards_sales':num_N_sales,"NonRewards_trans":num_N_trans,
                            },index=[0])
    
    df_cum_week=pd.DataFrame({"week_num":week_num,"week_start":date_min,"week_end":date_max,
                             "Rewards_sales":cum_R_sales,"Rewards_trans":cum_R_trans,"Rewards_shoppers":cum_R_shoppers,
                             'NonRewards_sales':cum_N_sales,"NonRewards_trans":cum_N_trans,
                            },index=[0])
    df_in_week=pd.merge(df_in_week,df_division_R,on="week_num",how="outer")
    df_in_week=pd.merge(df_in_week,df_division_N,on="week_num",how="outer")
    
    df_summary_InWeek_ty=df_summary_InWeek_ty.append(df_in_week)
    df_summary_CumQ_ty=df_summary_CumQ_ty.append(df_cum_week)
    print(datetime.datetime.now(),week_num,"done")

    
    

2020-01-24 16:26:15.835299 1 done
2020-01-24 16:28:51.552242 2 done
2020-01-24 16:33:05.378557 3 done


/home/jian/.local/lib/python3.6/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


2020-01-24 16:37:14.087468 4 done
2020-01-24 16:42:19.043969 5 done
2020-01-24 16:47:32.045603 6 done
2020-01-24 16:53:00.314159 7 done
2020-01-24 16:57:37.596783 8 done
2020-01-24 17:01:31.320648 9 done
2020-01-24 17:04:41.164403 10 done
2020-01-24 17:06:54.317776 11 done


In [37]:
df_summary_InWeek_ty=df_summary_InWeek_ty[df_summary_InWeek_ly.columns.tolist()]

writer=pd.ExcelWriter("./BL_projection_in_quarter_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_summary_InWeek_ty.to_excel(writer,"df_summary_InWeek_ty",index=False)
df_summary_CumQ_ty.to_excel(writer,"df_summary_CumQ_ty",index=False)

df_summary_InWeek_ly.to_excel(writer,"df_summary_InWeek_ly",index=False)
df_summary_CumQ_ly.to_excel(writer,"df_summary_CumQ_ly",index=False)

df_division_name.to_excel(writer,"df_division_name",index=False)
writer.save()

In [33]:
df_rewards_ids_ty.to_csv("./df_rewards_ids_ty.csv",index=False)
df_rewards_ids_ly.to_csv("./df_rewards_ids_ly.csv",index=False)

In [31]:
import sqlalchemy

In [34]:
df_rewards_ids_ty.head(2)

,customer_id_hashed,week_start,week_end
0,b445597aeb04ea0afe60321876a7580f52abaa8e4ef5b0...,2019-11-03,2019-11-09
1,23b14d85798fd5580f20d79e32824ad4cd6df31f5effe5...,2019-11-03,2019-11-09


In [35]:
df_rewards_ids_ly.head(3)

,customer_id_hashed,week_start,week_end
0,34e22ad0c4e216b4983bc2978770b3dbd7301213f8ebfd...,2018-11-04,2018-11-10
2,049f7c29a72fd252a880b32912fa49adc5dc55f3047193...,2018-11-04,2018-11-10
4,5216dbc5abd3e8d9057ef64926cc52ea99704484ee80b3...,2018-11-04,2018-11-10


In [36]:
df_rewards_ids_ty['week_start'].nunique()

11